<a href="https://colab.research.google.com/github/solislemuslab/colonization-squid/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install PyDrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [ ]:
data_path = '/content/gdrive/My Drive/squid/'
squid = pd.read_excel(data_path+'Dataset-S1.xlsx')
squid.shape

(3983, 31)

In [ ]:
SEED = 100
categories = [1,2,3,4,5,6]

In [ ]:
X, Y = squid.drop(['Category'], axis=1), squid.Category
xtrain_full, xtest, ytrain_full, ytest = \
  train_test_split(X, Y, test_size=0.2, random_state=SEED, stratify=Y)
xtrain, xval, ytrain, yval = \
  train_test_split(xtrain_full, ytrain_full, test_size=0.15, random_state=SEED, stratify=ytrain_full)
print(f'Shape: Training {xtrain.shape[0]}, Validation {xval.shape[0]}, Test {xtest.shape[0]}')

Shape: Training 2708, Validation 478, Test 797


In [ ]:
def getMetrics(label, pred):
  acc = accuracy_score(label, pred)
  # recall = recall_score(label, pred)
  f1 = f1_score(label, pred, average="weighted")
  confusion_mat = pd.DataFrame(confusion_matrix(label, pred), columns=categories, index=categories)

  print(f'Prediction Result:\nacc:{np.round(acc*100, 3)}%, f1 score:{np.round(f1,4)}.')
  print(confusion_mat)
  return acc, f1, confusion_matrix

# Dummy Classifier

In [ ]:
dummy_clf = DummyClassifier(strategy='most_frequent').fit(xtrain, ytrain)
dummy_pred_train = dummy_clf.predict(xtrain)
dummy_pred_val = dummy_clf.predict(xval)
dummy_pred_test = dummy_clf.predict(xtest)

In [ ]:
_, _, dummy_confusion_train = getMetrics(ytrain, dummy_pred_train)

Prediction Result:
acc:48.006%, f1 score:0.3114.
   1  2  3  4     5  6
1  0  0  0  0   106  0
2  0  0  0  0   309  0
3  0  0  0  0    78  0
4  0  0  0  0   657  0
5  0  0  0  0  1300  0
6  0  0  0  0   258  0


In [ ]:
_, _, dummy_confusion_val = getMetrics(yval, dummy_pred_val)

Prediction Result:
acc:47.908%, f1 score:0.3104.
   1  2  3  4    5  6
1  0  0  0  0   18  0
2  0  0  0  0   55  0
3  0  0  0  0   14  0
4  0  0  0  0  116  0
5  0  0  0  0  229  0
6  0  0  0  0   46  0


In [ ]:
_, _, dummy_confusion_test = getMetrics(ytest, dummy_pred_test)

Prediction Result:
acc:47.93%, f1 score:0.3106.
   1  2  3  4    5  6
1  0  0  0  0   31  0
2  0  0  0  0   91  0
3  0  0  0  0   23  0
4  0  0  0  0  194  0
5  0  0  0  0  382  0
6  0  0  0  0   76  0


# Baseline

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
drop_list = ['ID','Chromosome','Coordinate','Locus_Tag','Gene','Annotation']
xtrain_nums = xtrain.drop(drop_list, axis=1)
xval_nums = xval.drop(drop_list, axis=1)
xtest_nums = xtest.drop(drop_list, axis=1)

In [ ]:
rf_clf = RandomForestClassifier(random_state=SEED).fit(xtrain_nums, ytrain)
rf_pred_train = rf_clf.predict(xtrain_nums)
rf_pred_val = rf_clf.predict(xval_nums)
rf_pred_test = rf_clf.predict(xtest_nums)

In [ ]:
_, _, rf_confusion_train = getMetrics(ytrain, rf_pred_train)

Prediction Result:
acc:97.637%, f1 score:0.9779.
     1    2   3    4     5    6
1  106    0   0    0     0    0
2   56  253   0    0     0    0
3    0    1  77    0     0    0
4    6    1   0  650     0    0
5    0    0   0    0  1300    0
6    0    0   0    0     0  258


In [ ]:
_, _, rf_confusion_val = getMetrics(yval, rf_pred_val)

Prediction Result:
acc:88.703%, f1 score:0.8849.
    1   2   3    4    5   6
1  15   1   1    1    0   0
2   9  39   4    2    1   0
3   0   2  10    2    0   0
4   1   2   3  108    1   1
5   0   0   0    4  224   1
6   0   0   0    3   15  28


In [ ]:
_, _, _ = getMetrics(ytest, rf_pred_test)

Prediction Result:
acc:91.217%, f1 score:0.9108.
    1   2   3    4    5   6
1  27   0   1    2    1   0
2  11  72   3    3    1   1
3   0   5  12    6    0   0
4   6   6   1  180    0   1
5   0   0   0    2  379   1
6   0   0   0    2   17  57
